In [22]:
# --- CELL 1: IMPORT THU VIEN & CAI DAT ---
# !pip install selenium beautifulsoup4 pandas webdriver-manager openpyxl

import time
import pandas as pd
import random
import os
import platform
import subprocess
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

print("[INFO] Da import thu vien thanh cong.")

[INFO] Da import thu vien thanh cong.


In [23]:
# --- CELL 2: CONFIG DRIVER & POPUP BLOCKER ---

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--start-maximized")
    prefs = {"profile.default_content_setting_values.notifications": 2}
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

def close_popup_ads(driver):
    try:
        buttons = driver.find_elements(By.XPATH, "//a[contains(text(), 'Bữa khác nha')] | //button[contains(text(), 'Bữa khác nha')] | //div[contains(@class, 'cancel-button')] | //div[contains(@class, 'close-popup')]")
        for btn in buttons:
            if btn.is_displayed():
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(0.5)
                return True
    except: pass
    return False

print("[INFO] Da thiet lap Driver.")

[INFO] Da thiet lap Driver.


In [25]:
# --- CELL 3: HAM LAY LINK SAN PHAM ---
def get_product_links_deep(category_url, click_show_more_times=50):
    driver = setup_driver()
    driver.get(category_url)
    print(f"--- Dang quet danh muc: {category_url} ---")
    time.sleep(3)
    
    for i in range(click_show_more_times):
        try:
            close_popup_ads(driver)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1000);")
            time.sleep(2)
            
            load_more_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn-show-more, .button-load-more, .btn-view-more"))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_btn)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", load_more_btn)
            print(f"    [Page {i+1}/{click_show_more_times}] Da bam xem them...")
            time.sleep(3) 
        except TimeoutException:
            print(f"    [Stop] Het san pham hoac mang lag.")
            break 
        except Exception:
            close_popup_ads(driver)
            continue
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    product_links = set()
    all_links = soup.find_all('a', href=True)
    bad_words = ["sforum", "tin-tuc", "forum", "thuthuat", "hoi-dap", "gio-hang", "smember", "khuyen-mai"]
    
    for a in all_links:
        link = a['href']
        if "https" not in link:
            link = "https://cellphones.com.vn" + link if link.startswith("/") else "https://cellphones.com.vn/" + link
        if ".html" in link and not any(w in link for w in bad_words):
            product_links.add(link)
            
    driver.quit()
    return list(product_links)

print("[INFO] Da cap nhat ham get_product_links_deep.")

[INFO] Da cap nhat ham get_product_links_deep.


In [ ]:
# --- CELL 4: HAM CAO REVIEW (FIX LOI FORUM) ---
def crawl_reviews_from_url(driver, product_url, max_pages=20):
    data = []
    try:
        driver.get(product_url)
        time.sleep(3)
        close_popup_ads(driver)

        original_window = driver.current_window_handle
        has_switched_tab = False

        # 1. CHIEN THUAT CHUYEN TRANG (CHI BAM NUT TRONG BOX REVIEW)
        try:
            # QUAN TRONG: Chi tim nut "Xem tat ca" nam trong khoi .boxReview hoac .block-review
            # De tranh bam nham nut cua muc Tin tuc/Sforum
            view_all_btn = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'boxReview') or contains(@class, 'block-review')]//a[contains(text(), 'Xem tất cả') or contains(@class, 'btn-view-all')]"))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", view_all_btn)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", view_all_btn)
            time.sleep(3)

            # Check tab moi
            if len(driver.window_handles) > 1:
                driver.switch_to.window(driver.window_handles[-1])
                has_switched_tab = True
                close_popup_ads(driver)
        except: 
            pass 

        # 2. CLICK XEM THEM
        click_count = 0
        while click_count < max_pages:
            try:
                load_more = WebDriverWait(driver, 4).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".load-more, .button__view-more-review, .btn-show-more"))
                )
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", load_more)
                time.sleep(2)
                click_count += 1
                if click_count % 3 == 0: close_popup_ads(driver)
            except:
                break 
        
        # 3. PARSE
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        p_name = "Unknown"
        try: p_name = soup.select_one("h1").get_text(strip=True)
        except: pass

        items = soup.select(".boxReview-comment-item")
        for item in items:
            try:
                name = item.select_one(".block-info__name .name").get_text(strip=True)
                content = item.select_one(".comment-content p").get_text(strip=True)
                
                rating_block = item.select_one(".item-review-rating__star")
                stars = 0
                if rating_block:
                    stars = len(rating_block.select(".icon.is-active"))
                    if stars == 0: 
                         stars = len(rating_block.select("svg path[fill='#FF9F00']"))
                         if stars == 0: stars = 5
                
                # Loc bot
                if len(content.split()) < 2 or "?" in content: continue

                data.append({
                    'product': p_name, 
                    'user': name,
                    'rating': stars,
                    'content': content,
                    'source': product_url
                })
            except: continue
        
        if has_switched_tab:
            driver.close()
            driver.switch_to.window(original_window)
            
        return data

    except Exception:
        try:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        except: pass
        return []

print("[INFO] Da cap nhat ham crawl_reviews_from_url.")

[INFO] Da cap nhat ham crawl_reviews_from_url.


In [26]:
# --- CELL 5: QUET LINK SAN PHAM ---
CATEGORIES = [
    "https://cellphones.com.vn/mobile.html", 
    "https://cellphones.com.vn/laptop.html", 
    "https://cellphones.com.vn/tablet.html"
]

print("BAT DAU QUET LINK SAN PHAM...")
product_pool = []

for cat in CATEGORIES:
    links = get_product_links_deep(cat, click_show_more_times=50) 
    product_pool.extend(links)

product_pool = list(set(product_pool))
random.shuffle(product_pool)

print(f"\n[KET QUA] Tong cong tim thay {len(product_pool)} link san pham.")

BAT DAU QUET LINK SAN PHAM...
--- Dang quet danh muc: https://cellphones.com.vn/mobile.html ---
    [Page 1/50] Da bam xem them...
    [Page 2/50] Da bam xem them...
    [Page 3/50] Da bam xem them...
    [Page 4/50] Da bam xem them...
    [Page 5/50] Da bam xem them...
    [Page 6/50] Da bam xem them...
    [Page 7/50] Da bam xem them...
    [Page 8/50] Da bam xem them...
    [Page 9/50] Da bam xem them...
    [Page 10/50] Da bam xem them...
    [Page 11/50] Da bam xem them...
    [Page 12/50] Da bam xem them...
    [Page 13/50] Da bam xem them...
    [Page 14/50] Da bam xem them...
    [Page 15/50] Da bam xem them...
    [Page 16/50] Da bam xem them...
    [Page 17/50] Da bam xem them...
    [Page 18/50] Da bam xem them...
    [Page 19/50] Da bam xem them...
    [Page 20/50] Da bam xem them...
    [Page 21/50] Da bam xem them...
    [Page 22/50] Da bam xem them...
    [Page 23/50] Da bam xem them...
    [Page 24/50] Da bam xem them...
    [Page 25/50] Da bam xem them...
    [Page 26/

In [29]:
# --- CELL 6: CHAY CAO DU LIEU ---
all_data = [] 
driver = setup_driver()
TARGET = 7500 # Muc tieu 7500 dong

print(f"Bat dau cao review tu {len(product_pool)} link san pham...")

for idx, link in enumerate(product_pool):
    if len(all_data) >= TARGET: 
        print(f"DA DU {TARGET} DONG DU LIEU. DUNG LAI.")
        break
    
    print(f"[{idx+1}/{len(product_pool)}] {link}")
    
    # Goi ham cao tu Cell 4
    revs = crawl_reviews_from_url(driver, link, max_pages=15)
    
    if revs:
        all_data.extend(revs)
        print(f"   -> +{len(revs)} dong. Tong: {len(all_data)}")
    else:
        print("   -> Khong co data.")

    # Checkpoint
    if len(all_data) > 0 and len(all_data) % 500 < 50:
         try:
             pd.DataFrame(all_data).to_excel('checkpoint.xlsx', index=False)
             print("   [INFO] Da luu checkpoint.")
         except: pass

driver.quit()
print("Hoan thanh qua trinh cao.")

Bat dau cao review tu 1703 link san pham...
[1/1703] https://cellphones.com.vn/dien-thoai-honor-play-10-4gb-128gb.html
   -> Khong co data.
[2/1703] https://cellphones.com.vn/laptop-asus-zenbook-14-ux3405ca-pz188ws.html
   -> Khong co data.
[3/1703] https://cellphones.com.vn/samsung-galaxy-a50s.html
   -> +4 dong. Tong: 4
   [INFO] Da luu checkpoint.
[4/1703] https://cellphones.com.vn/laptop-dell-alienware-m18.html
   -> Khong co data.
   [INFO] Da luu checkpoint.
[5/1703] https://cellphones.com.vn/laptop-msi-crosshair-16-hx-d2xwfkg-078vn.html
   -> +8 dong. Tong: 12
   [INFO] Da luu checkpoint.
[6/1703] https://cellphones.com.vn/xiaomi-poco-f4.html
   -> +7 dong. Tong: 19
   [INFO] Da luu checkpoint.
[7/1703] https://cellphones.com.vn/dien-thoai-tecno-spark-30.html
   -> Khong co data.
   [INFO] Da luu checkpoint.
[8/1703] https://cellphones.com.vn/xiaomi-poco-c65.html
   -> Khong co data.
   [INFO] Da luu checkpoint.
[9/1703] https://cellphones.com.vn/ipad-mini-7-256gb.html
   -> Kho

In [30]:
# --- CELL 7: LUU FILE EXCEL ---
# 1. Tao DataFrame
final_df = pd.DataFrame(all_data)

if not final_df.empty:
    # 2. Xu ly so bo (Xoa trung lap)
    final_df.drop_duplicates(subset=['content'], inplace=True)
    
    # --- HAM NHAN DIEN THUONG HIEU ---
    def detect_brand(name):
        n = str(name).lower()
        # Nhom Apple
        if 'apple' in n or 'iphone' in n or 'ipad' in n or 'macbook' in n or 'mac' in n: return 'Apple'
        
        # Nhom Laptop/PC pho bien
        if 'dell' in n or 'alienware' in n or 'vostro' in n or 'inspiron' in n: return 'Dell'
        if 'hp' in n or 'pavilion' in n or 'victus' in n: return 'HP'
        if 'lenovo' in n or 'thinkpad' in n or 'ideapad' in n or 'legion' in n: return 'Lenovo'
        if 'acer' in n or 'nitro' in n or 'aspire' in n: return 'Acer'
        if 'msi' in n: return 'MSI'
        if 'surface' in n or 'microsoft' in n: return 'Microsoft'
        if 'lg' in n or 'gram' in n: return 'LG'
        if 'gigabyte' in n: return 'Gigabyte'
        
        # Nhom Dien thoai/Tablet pho bien
        if 'samsung' in n or 'galaxy' in n: return 'Samsung'
        if 'xiaomi' in n or 'redmi' in n or 'poco' in n: return 'Xiaomi'
        if 'oppo' in n or 'reno' in n: return 'Oppo'
        if 'asus' in n or 'rog' in n or 'tuf' in n or 'zenbook' in n: return 'Asus'
        if 'vivo' in n: return 'Vivo'
        if 'realme' in n: return 'Realme'
        if 'nokia' in n: return 'Nokia'
        if 'huawei' in n: return 'Huawei'
        if 'oneplus' in n: return 'OnePlus'
        if 'tecno' in n: return 'Tecno'
        
        return 'Other'

    # --- HAM PHAN LOAI SAN PHAM (MOBILE / LAPTOP / TABLET) ---
    def detect_category(name):
        n = str(name).lower()
        # Uu tien nhan dien Laptop truoc
        if 'laptop' in n or 'macbook' in n or 'thinkpad' in n or 'notebook' in n:
            return 'Laptop'
        # Tiep theo la Tablet
        if 'ipad' in n or 'tablet' in n or 'máy tính bảng' in n or 'tab' in n or 'pad' in n:
            return 'Tablet'
        # Con lai mac dinh la Mobile
        return 'Mobile'

    # Ap dung 2 ham tren (KEY 'product' da dong bo voi Cell 4)
    final_df['brand'] = final_df['product'].apply(detect_brand)
    final_df['category'] = final_df['product'].apply(detect_category)
    
    # 3. Luu file
    filename = 'cellphones_reviews_final.xlsx'
    current_dir = os.getcwd()
    full_path = os.path.join(current_dir, filename)
    
    try:
        final_df.to_excel(full_path, index=False)
        print(f"\n[SUCCESS] DA LUU FILE THANH CONG!")
        print(f"Vi tri file: {full_path}")
        print(f"Tong so dong: {len(final_df)}")
        print("-" * 30)
        print("Thong ke so luong theo loai:")
        print(final_df['category'].value_counts())
        
        # 4. TU DONG MO FILE (Auto Open)
        print("\n[INFO] Dang tu dong mo file Excel...")
        if platform.system() == 'Darwin':       # macOS
            subprocess.call(('open', full_path))
        elif platform.system() == 'Windows':    # Windows
            os.startfile(full_path)
        else:                                   # Linux 
            subprocess.call(('xdg-open', full_path))
            
    except Exception as e:
        print(f"[ERROR] Khong the luu hoac mo file. Loi: {e}")
        # Backup ra CSV neu loi Excel
        final_df.to_csv('backup_data.csv', index=False)
else:
    print("[WARNING] Khong co du lieu nao duoc cao.")


[SUCCESS] DA LUU FILE THANH CONG!
Vi tri file: c:\Users\ASPIRE 7\OneDrive\Tài liệu\UEH\KỲ V - HKC 2025\NLP\Final\cellphones_reviews_final.xlsx
Tong so dong: 3472
------------------------------
Thong ke so luong theo loai:
category
Mobile    2730
Laptop     472
Tablet     270
Name: count, dtype: int64

[INFO] Dang tu dong mo file Excel...
